In [1]:
# import all the required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict,OrderedDict
from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_distances
from operator import itemgetter
from scipy.spatial.distance import cosine
import json
import pickle

In [2]:
# reading data from dataset
df = pd.read_csv('combine.csv')
# breif about the columns in the dataset
df.head()

,id,phoneNo,location,occupation,certification,age,gender,type,availabilityPreference,availability,minimumWage,experience,clientsAttended,aadharCard,doorstepService,references,liscenced,shoppingliscence
0,1,9326523642,Ambegaon,Accountant,true,26,Female,Full Time,Afternoon,Whole Day,9500,8,49,065660562714,no,false,false,true
1,2,7029738484,Ambegaon,Brick mason,true,36,Male,Part Time,Morning,Morning,4000,4,25,394728990924,no,false,false,true
2,3,8386070476,Ambegaon,Carpenter,true,31,Male,Part Time,Evening,Evening,5500,8,42,777556713881,yes,true,false,false
3,4,8208265762,Ambegaon,Cashier,false,20,Female,Part Time,Evening,Evening,2500,2,13,938143072796,no,true,false,true
4,5,9127458171,Ambegaon,Childcare Worker,true,33,Female,Full Time,Evening,Whole Day,2000,3,28,112614367501,yes,true,false,true


In [3]:
df = df.drop(['phoneNo','id','availabilityPreference','aadharCard','liscenced','shoppingliscence'],axis=1)
df.dropna(inplace=True)
# df.head()

In [4]:
def classes_maker():
    for i in df.columns:
        le = preprocessing.LabelEncoder()
        le.fit(df[i])
        df[i] = le.transform(df[i])
classes_maker()

In [5]:
occupations=defaultdict(list)
def all_occupations_in_a_location():
    for index,row in df.iterrows():
        occupations[row['location']].append((index,row['occupation']))
        
    for key, values in occupations.items():
        t_set = set(occupations.get(key))
        occupations[key] = list(t_set)
all_occupations_in_a_location()

In [6]:
with open('occupations.txt','w') as f:
    f.write(str(pickle.dumps(occupations)))
print("Occupation dict has been pickled.")

Occupation dict has been pickled.


In [ ]:
dist = np.ndarray(shape=(np.shape(df)[0],np.shape(df)[0]))
def cos_func(index1,index2):
    return cosine_distances(df.loc[index1].values.reshape(1, -1),df.loc[index2].values.reshape(1, -1))

def cos_helper(indexes):
    for i in indexes:
        for j in indexes:
            if i != j:
                dist[i][j] = cos_func(i,j)

def commonities():
        for k, v in occupations.items(): # location iterator
            type(occupations[k]) # list type
            temp_list = occupations[k]
            indexes = []
            temp_list.sort(key=itemgetter(1)) # sort list according to occupation
            
            for i in range(len(occupations[k])):
                if i+1 < len(occupations[k]):
                    if temp_list[i][1] == temp_list[i+1][1]:
                        indexes.append(temp_list[i][0])
            cos_helper(indexes)
                
commonities()
print('Distances calculated')

In [ ]:
plt.plot(dist)
plt.show()